In [1]:
import pandas as pd
import numpy as np
import os
import random
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.datasets import dump_svmlight_file
import xgboost as xgb 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, SelectFromModel
from sklearn.ensemble import ExtraTreesRegressor
from scipy.stats import pearsonr

### Loading Data

In [24]:
X = pd.read_csv("X_train.csv",
                delimiter=',',
                index_col = 'id')
X.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,14168.823171,10514.380717,3316.149698,94230.695124,102.386606,92.677127,11108.748199,10866.505510,10837.622093,10.227734,...,NaN,12352.094085,846.014651,105.132144,102.112809,2090.004260,2.691845,1234.374109,1000.784475,9285.751272
1.0,17757.037554,NaN,4101.016273,92959.527633,NaN,99.855168,10013.959449,10826.607494,10076.101597,11.436970,...,NaN,16198.071494,776.084467,106.385590,103.472030,2474.051881,2.287976,NaN,1012.626705,11750.284764
2.0,14226.656663,11029.642499,NaN,124055.600561,100.542483,92.860892,NaN,10492.342868,NaN,10.810076,...,10329.704431,13976.063780,737.040332,103.671234,109.458246,2656.083281,2.843706,888.353607,1048.810385,9553.922728
3.0,8766.012436,7384.202998,2147.308418,100157.719990,104.855061,101.929026,10050.049932,10499.521099,10525.030989,10.092109,...,10008.251395,6212.127347,329.044233,105.084488,104.858546,1097.785204,2.732257,927.752967,1048.357330,NaN
4.0,13801.016418,13269.493652,3408.316953,92048.527786,103.759758,95.789235,9667.353978,10750.783106,10618.800750,12.006773,...,10095.782015,13772.061493,NaN,NaN,100.369834,2693.053231,2.702908,1471.354073,1071.284484,9423.533063


In [25]:
y = pd.read_csv("y_train.csv",
                delimiter=',',
                index_col='id')
# y = pd.Series(y).reset_index()
y.head()

,y
id,
0.0,74.0
1.0,51.0
2.0,70.0
3.0,52.0
4.0,85.0


### Imputing missing values

In [26]:

#we use knn
imputer = KNNImputer(n_neighbors=20)
X_imp = imputer.fit_transform(X)

### Normalize data

In [27]:
X_stdised = sklearn.preprocessing.StandardScaler().fit(X_imp).transform(X_imp)


###  Feature Selection/Dimensity reduction

In [28]:
#PCA reduction of dimensions
pca = sklearn.decomposition.PCA(n_components=554, random_state=42)
X_reduced = pca.fit_transform(X_stdised)

In [ ]:
#Umap reduction
import umap


reducer = umap.UMAP(random_state=42)
X_reduced = reducer.fit(digits.data)


### Hyperparameter tuning

In [29]:
def XGB_Never(n_estimators, max_depth, sampling_method, reg_alpha, reg_lambda):
  regressor_xgb = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, sampling_method=sampling_method, reg_alpha=reg_alpha*0.1, reg_lambda=reg_lambda*0.1, random_state=42)
  regressor_xgb.fit(X_reduced, y)
  print('XGB (train): ' + str(r2_score(y, regressor_xgb.predict(X_reduced))))

  return r2_score(y, regressor_xgb.predict(X_reduced))

In [30]:
import nevergrad as ng


n_estimators = ng.p.TransitionChoice(range(1,101))

#start = 0.1
#stop = 0.7
#step = 0.01
#float_range = [x / 100 for x in range(int(start * 100), int(stop * 100), int(step * 100))]


#eta = ng.p.TransitionChoice(float_range) # eta
#print(eta)
max_depth = ng.p.TransitionChoice(range(4,11))
sampling_method = ng.p.Choice(['uniform', 'gradient_based'])
reg_alpha =  ng.p.TransitionChoice(range(0,10, 1))
reg_lambda =  ng.p.TransitionChoice(range(0,10, 1))

params = ng.p.Instrumentation(n_estimators, max_depth, sampling_method, reg_alpha, reg_lambda)
optimizer = ng.optimizers.TwoPointsDE(parametrization=params, budget=10000)
bestX = optimizer.minimize(XGB_Never, batch_mode=False)

/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dty

XGB (train): 0.9617109710178833


/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dty

XGBoostError: [14:06:29] /workspace/src/tree/hist/sampler.h:51: Check failed: param.sampling_method == TrainParam::kUniform (1 vs. 0) : Only uniform sampling is supported, gradient-based sampling is only support by GPU Hist.
Stack trace:
  [bt] (0) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x7001aa) [0x7fc28ae031aa]
  [bt] (1) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x74a9b7) [0x7fc28ae4d9b7]
  [bt] (2) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x460609) [0x7fc28ab63609]
  [bt] (3) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x4610fc) [0x7fc28ab640fc]
  [bt] (4) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x4c4e87) [0x7fc28abc7e87]
  [bt] (5) /home/linuxbrew/

In [ ]:
bestX.value